In [1]:
import numpy as np
from machine_learning.xgb_def import XGBOptimizer
from machine_learning.reglog_def import REGLOGOptimizer
import pandas as pd
from opt_funct.data_fuct import functions
from sklearn.model_selection import train_test_split
from machine_learning.SVM_defopt import SVMOptimizer
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from opt_funct.data_fuct import STRATEGY_XGBOOST_BUY
from opt_funct.data_fuct import STRATEGY_REGLOG_BUY
from opt_funct.data_fuct import STRATEGY_SVC_BUY
from opt_funct.data_fuct import STRATEGY_XGBOOST_REGLOG_BUY
from opt_funct.data_fuct import STRATEGY_XGBOOST_SVC_BUY
from opt_funct.data_fuct import STRATEGY_REGLOG_SVC_BUY
from opt_funct.data_fuct import STRATEGY_XGBOOST_REGLOG_SVC_BUY
from opt_funct.data_fuct import STRATEGY_XGBOOST_SELL
from opt_funct.data_fuct import STRATEGY_REGLOG_SELL
from opt_funct.data_fuct import STRATEGY_SVC_SELL
from opt_funct.data_fuct import STRATEGY_XGBOOST_REGLOG_SELL
from opt_funct.data_fuct import STRATEGY_XGBOOST_SVC_SELL
from opt_funct.data_fuct import STRATEGY_REGLOG_SVC_SELL
from opt_funct.data_fuct import STRATEGY_XGBOOST_REGLOG_SVC_SELL
import optuna
from opt_funct.data_fuct import Operation



/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = functions.clean_ds(pd.read_csv('./data/aapl_5m_train.csv'))
df

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,X_t-1,X_t-2,X_t-3,Pt_5,RSI,Y_BUY,Y_SELL
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0,NaN,NaN,NaN,131.448898,NaN,False,True
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0,132.809997,NaN,NaN,131.119995,NaN,False,True
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0,131.889999,132.809997,NaN,130.875000,NaN,False,True
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0,132.059997,131.889999,132.809997,130.509902,NaN,False,True
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0,132.250000,132.059997,131.889999,130.889999,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39574,19788,1672432800,0,2022-12-30 20:40:00,129.029998,129.130004,128.919998,129.054992,833767.0,129.039993,129.020004,128.910003,NaN,56.509324,False,False
39575,19789,1672433100,0,2022-12-30 20:45:00,129.059997,129.399993,129.020004,129.380004,2136910.0,129.054992,129.039993,129.020004,NaN,60.700323,False,False
39576,19790,1672433400,0,2022-12-30 20:50:00,129.375000,129.929992,129.330001,129.764404,2879267.0,129.380004,129.054992,129.039993,NaN,64.854367,False,False
39577,19791,1672433700,0,2022-12-30 20:55:00,129.764999,129.949996,129.619995,129.949996,3896812.0,129.764404,129.380004,129.054992,NaN,66.620919,False,False


**Entrenamos nuestros modelos para compra y venta**

In [3]:
# Buy
close_data = df[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_BUY']].dropna()
close_data_bien = close_data.copy()
x_buy = close_data_bien.drop(columns=['Timestamp','Y_BUY'])
y_buy = close_data_bien[['Y_BUY']]
x_train_buy, x_test_buy, y_train_buy, y_test_buy = train_test_split(x_buy, y_buy, test_size=0.3, shuffle=False)

close_data2 = df[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_SELL']].dropna()
close_data_bien2 = close_data2.copy()
x_sell = close_data_bien2.drop(columns=['Timestamp','Y_SELL'])
y_sell = close_data_bien2[['Y_SELL']]
x_train_sell, x_test_sell, y_train_sell, y_test_sell = train_test_split(x_sell, y_sell, test_size=0.3, shuffle=False)

In [4]:
# Escalado para el conjunto de compra
scaler_buy = StandardScaler()
x_train_buy_scaled = scaler_buy.fit_transform(x_train_buy)
x_test_buy_scaled = scaler_buy.transform(x_test_buy)

# Escalado para el conjunto de venta
scaler_sell = StandardScaler()
x_train_sell_scaled = scaler_sell.fit_transform(x_train_sell)
x_test_sell_scaled = scaler_sell.transform(x_test_sell)

In [5]:
# buy
xgboost_optimizer = XGBOptimizer(x_train_buy, y_train_buy, x_test_buy, y_test_buy)
xgboost_best_params_buy, best_model_xgboost_buy = xgboost_optimizer.xgb_optuna()

reg_log_optimizer = REGLOGOptimizer(x_train_buy, y_train_buy, x_test_buy, y_test_buy)
reg_log_best_params_buy, best_model_reg_log_buy = reg_log_optimizer.reglog_optuna()

SVC_optimizer = SVMOptimizer(x_train_buy_scaled, y_train_buy, x_test_buy_scaled, y_test_buy)
SVC_best_params_buy, best_model_SVC_buy = SVC_optimizer.svm_optuna()

# sell
xgboost_optimizer = XGBOptimizer(x_train_sell, y_train_sell, x_test_sell, y_test_sell)
xgboost_best_params_sell, best_model_xgboost_sell = xgboost_optimizer.xgb_optuna()

reg_log_optimizer = REGLOGOptimizer(x_train_sell, y_train_sell, x_test_sell, y_test_sell)
reg_log_best_params_sell, best_model_reg_log_sell = reg_log_optimizer.reglog_optuna()

SVC_optimizer = SVMOptimizer(x_train_sell_scaled, y_train_sell, x_test_sell_scaled, y_test_sell)
SVC_best_params_sell, best_model_SVC_sell = SVC_optimizer.svm_optuna()

[I 2024-03-10 22:19:07,930] A new study created in memory with name: no-name-dd3f08f1-abc0-4789-ac22-f52900c5e608
[I 2024-03-10 22:19:12,646] Trial 0 finished with value: 0.6788796703602651 and parameters: {'n_estimators': 95, 'max_depth': 77, 'max_leaves': 4, 'learning_rate': 1.818010317136557, 'booster': 'dart', 'gamma': 19.234181854177308, 'reg_alpha': 9.240096306051884, 'reg_lambda': 6.3559428446506185}. Best is trial 0 with value: 0.6788796703602651.
[I 2024-03-10 22:19:12,911] Trial 1 finished with value: 0.22922490470139772 and parameters: {'n_estimators': 28, 'max_depth': 96, 'max_leaves': 62, 'learning_rate': 1.1494992265450246, 'booster': 'dart', 'gamma': 10.73745211409105, 'reg_alpha': 1.010984018113449, 'reg_lambda': 4.8468049315322235}. Best is trial 0 with value: 0.6788796703602651.
[I 2024-03-10 22:19:13,696] Trial 2 finished with value: 0.6788796703602651 and parameters: {'n_estimators': 47, 'max_depth': 99, 'max_leaves': 29, 'learning_rate': 1.95939794298298, 'booster'

F1: 0.6788796703602651
Best hyperparameters: {'n_estimators': 95, 'max_depth': 77, 'max_leaves': 4, 'learning_rate': 1.818010317136557, 'booster': 'dart', 'gamma': 19.234181854177308, 'reg_alpha': 9.240096306051884, 'reg_lambda': 6.3559428446506185}
Execution time: 0.2090787370999654 minutes


[I 2024-03-10 22:19:23,679] A new study created in memory with name: no-name-4ff84728-1a05-4e25-9813-13f07d0e59a8
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-10 22:19:25,401] Trial 0 finished with value: 0.5857287303192258 and parameters: {'C': 0.3719817826502747, 'fit_intercept': True, 'l1_ratio': 0.9583798414018412}. Best is trial 0 with value: 0.5857287303192258.
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-10 22:19:27,205] Trial 1 finished with value: 0.5815241148185925 and parameters: 

F1 Score: 0.5857287303192258
Best hyperparameters: {'C': 0.3719817826502747, 'fit_intercept': True, 'l1_ratio': 0.9583798414018412}
Execution time: 0.30350979963938396 minutes


[I 2024-03-10 22:19:59,226] Trial 0 finished with value: 0.5127455755804514 and parameters: {'C': 4.4906514621249025, 'kernel': 'rbf', 'degree': 3, 'gamma': 'auto'}. Best is trial 0 with value: 0.5127455755804514.
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-10 22:20:15,314] Trial 1 finished with value: 0.6787703274671419 and parameters: {'C': 5.693401239517664, 'kernel': 'poly', 'degree': 4, 'gamma': 'scale'}. Best is trial 1 with value: 0.6787703274671419.
/home/rodo/.pyenv/versions/3.11.8/lib/python

...

F1: 0.4574408642205252
Best hyperparameters: {'n_estimators': 65, 'max_depth': 97, 'max_leaves': 18, 'learning_rate': 0.682293643065028, 'booster': 'gbtree', 'gamma': 4.736805796053508, 'reg_alpha': 0.07816957273647569, 'reg_lambda': 0.022508701080144897}
Execution time: 0.11034125487009684 minutes


[I 2024-03-10 22:22:34,812] Trial 0 finished with value: 0.3221957040572792 and parameters: {'C': 2.4865068250765643, 'fit_intercept': True, 'l1_ratio': 0.42462214236503826}. Best is trial 0 with value: 0.3221957040572792.
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-10 22:22:36,673] Trial 1 finished with value: 0.3221957040572792 and parameters: {'C': 4.705457017668767, 'fit_intercept': True, 'l1_ratio': 0.6841137421117771}. Best is trial 0 with value: 0.3221957040572792.
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1

F1 Score: 0.3397809368445584
Best hyperparameters: {'C': 9.128215433551444, 'fit_intercept': False, 'l1_ratio': 0.15634014935887108}
Execution time: 0.30711734692255654 minutes


[I 2024-03-10 22:23:05,753] Trial 0 finished with value: 0.4908580661620886 and parameters: {'C': 6.5944275749493215, 'kernel': 'sigmoid', 'degree': 3, 'gamma': 'scale'}. Best is trial 0 with value: 0.4908580661620886.
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-10 22:23:23,166] Trial 1 finished with value: 0.0059099600208586825 and parameters: {'C': 4.406921102493847, 'kernel': 'poly', 'degree': 5, 'gamma': 'scale'}. Best is trial 0 with value: 0.4908580661620886.
/home/rodo/.pyenv/versions/3.11.8/li

F1 Score: 0.49102814865209626
Best hyperparameters: {'C': 8.765646891437227, 'kernel': 'sigmoid', 'degree': 2, 'gamma': 'scale'}
Execution time: 2.5535186688105265 minutes


In [6]:
# Indicadores Buy
close_data['xg_boost_buy'] = best_model_xgboost_buy.predict(x_buy)
close_data['reg_log_buy'] = best_model_reg_log_buy.predict(x_buy)
close_data['SVC_buy'] = best_model_SVC_buy.predict(x_buy)
# Indicadores Sell
close_data = close_data.drop(columns=['Y_BUY','RSI','X_t-1','X_t-2','X_t-3'])
close_data['xg_boost_sell'] = best_model_xgboost_sell.predict(x_sell)
close_data['reg_log_sell'] = best_model_reg_log_sell.predict(x_sell)
close_data['SVC_sell'] = best_model_SVC_sell.predict(x_sell)
# Convertimos a boleanos
close_data['xg_boost_buy'] = np.where(close_data['xg_boost_buy']==1, True, False)
close_data['xg_boost_sell'] = np.where(close_data['xg_boost_sell']==1, True, False)
close_data

/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


,Timestamp,Close,xg_boost_buy,reg_log_buy,SVC_buy,xg_boost_sell,reg_log_sell,SVC_sell
27,1609778700,129.413299,True,False,True,False,True,True
28,1609779000,129.148498,True,False,True,False,True,True
29,1609779300,128.820007,True,False,True,False,True,True
30,1609779600,128.580001,True,False,True,False,True,True
31,1609779900,128.351699,True,False,True,False,True,True
...,...,...,...,...,...,...,...,...
39574,1672432800,129.054992,True,True,True,True,False,True
39575,1672433100,129.380004,True,True,True,False,False,True
39576,1672433400,129.764404,True,True,True,True,False,True
39577,1672433700,129.949996,True,True,True,True,False,True


In [7]:
df_tets = functions.clean_ds(pd.read_csv('./data/aapl_5m_test.csv'))

close_data_test = df_tets[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_BUY']].dropna()
cl_dat_test = close_data_test.copy()
x_buy_test = cl_dat_test.drop(columns=['Timestamp','Y_BUY'])

close_data2_test = df_tets[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_SELL']].dropna()
cl_dat2_test = close_data2_test.copy()
x_sell_test = cl_dat2_test.drop(columns=['Timestamp','Y_SELL'])

# Indicadores Buy
close_data_test['xg_boost_buy'] = best_model_xgboost_buy.predict(x_buy_test)
close_data_test['reg_log_buy'] = best_model_reg_log_buy.predict(x_buy_test)
close_data_test['SVC_buy'] = best_model_SVC_buy.predict(x_buy_test)
# Indicadores Sell
close_data_test = close_data_test.drop(columns=['Y_BUY','RSI','X_t-1','X_t-2','X_t-3'])
close_data_test['xg_boost_sell'] = best_model_xgboost_sell.predict(x_sell_test)
close_data_test['reg_log_sell'] = best_model_reg_log_sell.predict(x_sell_test)
close_data_test['SVC_sell'] = best_model_SVC_sell.predict(x_sell_test)
# Convertimos a boleanos
close_data_test['xg_boost_buy'] = np.where(close_data_test['xg_boost_buy']==1, True, False)
close_data_test['xg_boost_sell'] = np.where(close_data_test['xg_boost_sell']==1, True, False)
close_data_test


/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


,Timestamp,Close,xg_boost_buy,reg_log_buy,SVC_buy,xg_boost_sell,reg_log_sell,SVC_sell
27,1672764300,124.849998,True,False,True,False,True,True
28,1672764600,124.599899,True,False,True,False,True,True
29,1672764900,124.742500,True,False,True,False,True,True
30,1672765200,124.730003,True,False,True,False,True,True
31,1672765500,124.930000,True,False,True,False,True,True
...,...,...,...,...,...,...,...,...
19674,1703882400,192.500000,True,False,True,True,True,True
19675,1703882700,192.510299,True,False,True,True,True,True
19676,1703883000,192.369995,True,False,True,True,True,True
19677,1703883300,192.520004,True,False,True,True,True,True


In [ ]:
close_data_test.to_csv('./data/indicadores_test_5m.csv', index=False)

In [ ]:
close_data.keys()

**Mostramos nuestras combinaciones posibles de compra y venta**

In [ ]:
indicadores_compra = ['xg_boost_buy', 'reg_log_buy' , 'SVC_buy']
indicadores_venta = ['xg_boost_sell', 'reg_log_sell' , 'SVC_sell']

combinaciones_indicadores_compra = []
combinaciones_indicadores_venta = []

for i in range(1, len(indicadores_compra) + 1):
    combinaciones_indicadores_compra.extend(combinations(indicadores_compra, i))

for i in range(1, len(indicadores_venta) + 1):
    combinaciones_indicadores_venta.extend(combinations(indicadores_venta, i))
    
combinaciones_indicadores_compra, combinaciones_indicadores_venta

In [ ]:
cash = 1_000_000
active_operations = []
com = 0.00125  # comision en GBM
strategy_value = [1_000_000]


Calculamos la mejor combinacion para compra

In [ ]:
best_global_strategy = {'name': None, 'value': float('-inf')}

def optimize(trial):
    # Definición de los parámetros a optimizar
    stop_loss_long = trial.suggest_float('stop_loss_long', 0.00250, 0.05)
    take_profit_long = trial.suggest_float('take_profit_long', 0.00250, 0.05)
    n_shares = trial.suggest_int('n_shares', 10, 100)
    
    # Inicialización de la estrategia con los parámetros actuales
    xg_boost_buy = STRATEGY_XGBOOST_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    reglog_buy = STRATEGY_REGLOG_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    svc_buy = STRATEGY_SVC_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_reglog_buy = STRATEGY_XGBOOST_REGLOG_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_svc_buy = STRATEGY_XGBOOST_SVC_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    reglog_svc_buy = STRATEGY_REGLOG_SVC_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_reglog_svc_buy = STRATEGY_XGBOOST_REGLOG_SVC_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    strategy_values = {
        'xgboost_buy': xg_boost_buy.run_strategy(),
        'reglog_buy': reglog_buy.run_strategy(),
        'svc_buy': svc_buy.run_strategy(),
        'xgboost_reglog_buy': xgboost_reglog_buy.run_strategy(),
        'xgboost_svc_buy': xgboost_svc_buy.run_strategy(),
        'reglog_svc_buy': reglog_svc_buy.run_strategy(),
        'xgboost_reglog_svc_buy': xgboost_reglog_svc_buy.run_strategy()
    }

    # Encuentra el mayor valor de estrategia y su correspondiente estrategia
    best_strategy_name = max(strategy_values, key=strategy_values.get)
    best_strategy_value = strategy_values[best_strategy_name]

    if best_strategy_value > best_global_strategy['value']:
        best_global_strategy['name'] = best_strategy_name
        best_global_strategy['value'] = best_strategy_value

    
    # Retorna el valor de la mejor estrategia
    return best_strategy_value

In [ ]:
# Inicializar y ejecutar la optimización
study = optuna.create_study(direction='maximize')
study.optimize(optimize, n_trials=150, n_jobs=-1)

# Los mejores parámetros encontrados en el mejor trial
best_params = study.best_trial.params
best_value = study.best_trial.value

# Comparar con el mejor valor global previamente encontrado y el nombre de la estrategia
best_strategy_name = best_global_strategy['name']
best_strategy_value = best_global_strategy['value']

# Imprimir los resultados, incluido el nombre de la mejor estrategia global y su valor
print(f"Best buy overall strategy: {best_strategy_name} with value: {best_strategy_value}")
print("Best buy strategy parameters:", best_params)

Calculamos la mejor combinacion para venta

In [ ]:
best_global_sell_strategy = {'name': None, 'value': float('-inf')}

def optimize_sell(trial):
    # Definición de los parámetros a optimizar
    stop_loss_short = trial.suggest_float('stop_loss_short', 0.00250, 0.05)
    take_profit_short = trial.suggest_float('take_profit_short', 0.00250, 0.05)
    n_shares = trial.suggest_int('n_shares', 10, 100)
    
    # Inicialización de la estrategia con los parámetros actuales
    xg_boost_sell = STRATEGY_XGBOOST_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    reglog_sell = STRATEGY_REGLOG_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    svc_sell = STRATEGY_SVC_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_reglog_sell = STRATEGY_XGBOOST_REGLOG_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_svc_sell = STRATEGY_XGBOOST_SVC_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    reglog_svc_sell = STRATEGY_REGLOG_SVC_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_reglog_svc_sell = STRATEGY_XGBOOST_REGLOG_SVC_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    strategy_values = {
        'xgboost_sell': xg_boost_sell.run_strategy(),
        'reglog_sell': reglog_sell.run_strategy(),
        'svc_sell': svc_sell.run_strategy(),
        'xgboost_reglog_sell': xgboost_reglog_sell.run_strategy(),
        'xgboost_svc_sell': xgboost_svc_sell.run_strategy(),
        'reglog_svc_sell': reglog_svc_sell.run_strategy(),
        'xgboost_reglog_svc_sell': xgboost_reglog_svc_sell.run_strategy()
    }

    # Encuentra el mayor valor de estrategia y su correspondiente estrategia
    best_strategy_name = max(strategy_values, key=strategy_values.get)
    best_strategy_value = strategy_values[best_strategy_name]

    if best_strategy_value > best_global_sell_strategy['value']:
        best_global_sell_strategy['name'] = best_strategy_name
        best_global_sell_strategy['value'] = best_strategy_value

    
    # Retorna el valor de la mejor estrategia
    return best_strategy_value

In [ ]:
# Inicializar y ejecutar la optimización
study = optuna.create_study(direction='maximize')
study.optimize(optimize_sell, n_trials=150, n_jobs=-1)

# Los mejores parámetros encontrados en el mejor trial
best_params = study.best_trial.params
best_value = study.best_trial.value

# Comparar con el mejor valor global previamente encontrado y el nombre de la estrategia
best_strategy_name = best_global_sell_strategy['name']
best_strategy_value = best_global_sell_strategy['value']

# Imprimir los resultados, incluido el nombre de la mejor estrategia global y su valor
print(f"Best sell overall strategy: {best_strategy_name} with value: {best_strategy_value}")
print("Best sell strategy parameters:", best_params)

In [ ]:
import matplotlib.pyplot as plt





# Parámetros óptimos
stop_loss_long = 0.0317407648006915
take_profit_long = 0.04998058375999087
stop_loss_short = 0.049689221836262565
take_profit_short = 0.021530456551386253
n_shares_long = 97
n_shares_short = 99

# Variables de la estrategia
cash = 1_000_000
active_operations = []
com = 0.00125
strategy_value = [1_000_000]

class Operation:
    def __init__(self, operation_type, bought_at, timestamp, n_shares,
                 stop_loss,take_profit):
        self.operation_type = operation_type
        self.bought_at = bought_at
        self.timestamp = timestamp
        self.n_shares = n_shares
        self.sold_at =  None
        self.stop_loss = stop_loss
        self.take_profit = take_profit

# Ejecutar la estrategia
for i, row in close_data.iterrows():
    # Cerrar operaciones activas
    temp_operations = []
    for op in active_operations:
        if op.operation_type == 'Long':
            if op.stop_loss > row.Close:  # Cerrar operaciones perdedoras
                cash += row.Close * op.n_shares * (1 - com)
            elif op.take_profit < row.Close:  # Cerrar operaciones con ganancias
                cash += row.Close * op.n_shares * (1 - com)
            else:
                temp_operations.append(op)
        elif op.operation_type == 'Short':
            if op.stop_loss < row.Close:  # Cerrar operaciones perdedoras
                cash -= row.Close * op.n_shares * (1 + com)
            elif op.take_profit > row.Close:  # Cerrar operaciones con ganancias
                cash -= row.Close * op.n_shares * (1 + com)
            else:
                temp_operations.append(op)
    active_operations = temp_operations

    if cash >= row['Close'] * n_shares_long * (1 + com):
        if row['SVC_buy']:
            active_operations.append(Operation(operation_type='Long',
                                                    bought_at=row['Close'],
                                                    timestamp=row.Timestamp,
                                                    n_shares= n_shares_long,
                                                    stop_loss=(row['Close'] * stop_loss_long),
                                                    take_profit=row['Close'] * (1 + take_profit_long)))
            cash -= row['Close'] * n_shares_long * (1 + com)

    if cash >= row['Close'] * n_shares_short * (1 + com):

        if row['reg_log_sell']:
            active_operations.append(Operation(operation_type='Short',
                                                    bought_at=row['Close'],
                                                    timestamp=row.Timestamp,
                                                    n_shares=n_shares_short,
                                                    stop_loss=row['Close'] * (1 + stop_loss_short),
                                                    take_profit=row['Close'] * take_profit_short))

            cash += (op.bought_at - row['Close']) * (1- com) * n_shares_short

    # Valor de la estrategia
    total_value = len(active_operations) * row.Close * (n_shares_short + n_shares_long)
    strategy_value.append(cash + total_value)

# Graficar el valor de la estrategia a lo largo del tiempo
plt.figure(figsize=(12, 6))
plt.plot(strategy_value)
plt.title('Valor de la estrategia combinada')
plt.xlabel('Período')
plt.ylabel('Valor')
plt.show()
print("Valor de la estrategia: ", strategy_value[-1])


In [ ]:
# Parámetros óptimos
stop_loss_long = 0.08019149204913106
take_profit_long = 0.23163621752025085
stop_loss_short = 0.18804045696314062
take_profit_short = 0.0357906315299027
n_shares_long = 96
n_shares_short = 67

# Variables de la estrategia
cash = 1_000_000
active_operations = []
com = 0.00125
strategy_value = [1_000_000]

class Operation:
    def __init__(self, operation_type, bought_at, timestamp, n_shares,
                 stop_loss,take_profit):
        self.operation_type = operation_type
        self.bought_at = bought_at
        self.timestamp = timestamp
        self.n_shares = n_shares
        self.sold_at =  None
        self.stop_loss = stop_loss
        self.take_profit = take_profit

# Ejecutar la estrategia
for i, row in close_data.iterrows():
    # Cerrar operaciones activas
    temp_operations = []
    for op in active_operations:
        if op.operation_type == 'Long':
            if op.stop_loss > row.Close:  # Cerrar operaciones perdedoras
                cash += row.Close * op.n_shares * (1 - com)
            elif op.take_profit < row.Close:  # Cerrar operaciones con ganancias
                cash += row.Close * op.n_shares * (1 - com)
            else:
                temp_operations.append(op)
        elif op.operation_type == 'Short':
            if op.stop_loss < row.Close:  # Cerrar operaciones perdedoras
                cash -= row.Close * op.n_shares * (1 + com)
            elif op.take_profit > row.Close:  # Cerrar operaciones con ganancias
                cash -= row.Close * op.n_shares * (1 + com)
            else:
                temp_operations.append(op)
    active_operations = temp_operations

    if cash >= row['Close'] * n_shares_long * (1 + com):
                if row['SVC_buy']:
                    active_operations.append(Operation(operation_type='Long',
                                                            bought_at=row['Close'],
                                                            timestamp=row.Timestamp,
                                                            n_shares= n_shares_long,
                                                            stop_loss=(row['Close'] * stop_loss_long),
                                                            take_profit=row['Close'] * (1 + take_profit_long)))
                    cash -= row['Close'] * n_shares_long * (1 + com)

    if cash >= row['Close'] * n_shares_short * (1 + com):

                if row['xg_boost_sell']:
                    active_operations.append(Operation(operation_type='Short',
                                                            bought_at=row['Close'],
                                                            timestamp=row.Timestamp,
                                                            n_shares=n_shares_short,
                                                            stop_loss=row['Close'] * (1 + stop_loss_short),
                                                            take_profit=row['Close'] * take_profit_short))

                    cash += (op.bought_at - row['Close']) * (1- com) * n_shares_short

    # Valor de la estrategia
    total_value = len(active_operations) * row.Close * (n_shares_short + n_shares_long)
    strategy_value.append(cash + total_value)

# Graficar el valor de la estrategia a lo largo del tiempo
plt.figure(figsize=(12, 6))
plt.plot(strategy_value)
plt.title('Valor de la estrategia combinada')
plt.xlabel('Período')
plt.ylabel('Valor')
plt.show()
print("Valor de la estrategia: ", strategy_value[-1])